# The CLLJ is increasingly important as a predictor of early-season rainfall

Plot correlation maps between 700-hPa relative humidity and regionally-averaged indices of absolute sea surface temperature (SST), relative SST, and the Caribbean Low-level Jet (CLLJ). We compare the correlation maps between 1979-2001 and 2002-2024. 

In [ ]:
# Package imports
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
import cmaps as ncl_cm

from cartopy.mpl.geoaxes import GeoAxes
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.ticker as mticker


In [ ]:
# Figure format defaults
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['figure.titlesize'] = 15
plt.rcParams["axes.titlesize"] = 15

# Define AxesGrid parameters to set map projections and orientation
projection = ccrs.PlateCarree(central_longitude=-180)
mproj = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(projection=projection))

## Retrieve seasonal indices and gridded data

In [ ]:
# Relative humidity at 700 hPa
r700 = xr.open_dataset('../data/gridded_data/era5_interpolated_sst_MJJ_trend_results_1979to2024.nc')

# Seasonal predictors
sst_index = pd.read_csv('../data/seasonal_indices/absolute_sst_index_MJJ_1979to2024.csv', index_col=0, parse_dates=True)
rsst_index = pd.read_csv('../data/seasonal_indices/relative_sst_index_MJJ_1979to2024.csv', index_col=0, parse_dates=True)
cllj_index = pd.read_csv('../data/seasonal_indices/cllj_index_MJJ_1979to2024.csv', index_col=0, parse_dates=True)

# Separate into two time periods

## Calculate the correlation at each grid point

In [ ]:
## Define functions for pointwise correlation and climate index calculation
from scipy import stats

def gridpoint_correlation(y, x):
  """
  Performs a pointwise correlation between a time series x and a 3D (time, lat, lon) field.

  Args:
      y (np.ndarray): The dependent variable field at a particular grid point.
      x (np.ndarray): The independent variable time series.
  """

  slope, pval = np.zeros_like(y[0,:,:]), np.zeros_like(y[0,:,:])

  for ilat in range(y.shape[1]):
    for ilon in range(y.shape[2]):
      y_ij = y[:, ilat, ilon]
      x_ij = x

      # Perform linear regression
      PearsonRResult = stats.pearsonr(x_ij, y_ij)
      slope[ilat, ilon], pval[ilat, ilon] = PearsonRResult[0], PearsonRResult[1]

  return slope, pval

def climate_indices_calc(da, period, sel_months, sel_lats, sel_lons):
  da_subset = da.sel(time=da.time.dt.year.isin(period))

  if da.name in ['precip', 'sst']:
    da_subset = da_subset.sel(time = da_subset.time.dt.month.isin(sel_months),
                              lat=sel_lats,
                              lon=sel_lons).groupby('time.year').mean(['time', 'lat', 'lon'])
  else:
    da_subset = da_subset.sel(time = da_subset.time.dt.month.isin(sel_months),
                              latitude=sel_lats,
                              longitude=sel_lons).groupby('time.year').mean(['time', 'latitude', 'longitude'])
  return da_subset

In [ ]:
# Calculate correlation maps for each time period
sst_p1_slope, sst_p1_pval = gridpoint_correlation(rh700_mjj_period1.values, sst_mj_period1.values)
sst_p2_slope, sst_p2_pval = gridpoint_correlation(rh700_mjj_period2.values, sst_mj_period2.values)

rsst_p1_slope, rsst_p1_pval = gridpoint_correlation(rh700_mjj_period1.values, rsst_mj_period1.values)
rsst_p2_slope, rsst_p2_pval = gridpoint_correlation(rh700_mjj_period2.values, rsst_mj_period2.values)

cllj_p1_slope, cllj_p1_pval = gridpoint_correlation(rh700_mjj_period1.values, cllj_mj_period1.values)
cllj_p2_slope, cllj_p2_pval = gridpoint_correlation(rh700_mjj_period2.values, cllj_mj_period2.values)

## Plot correlation maps

In [ ]:
fig = plt.figure(figsize=(12, 12), dpi=300)

# Define extent
extent = [-100, -20, 5, 35]

# Define levels for plotting correlation coefficients (adjust as needed)
levels_corr = np.arange(-1, 1.1, 0.1)

# Variables and their correlation results and p-values for each period
# Assuming variables are Precipitation, SST, and a third variable (replace with actual variables)
row_variables = ['SST', 'RSST', 'CLLJ'] # Replace 'Variable 3' with your actual variable
subplot_titles = ['1979-2001', '2002-2024']
subplot_labels = ['a', 'b', 'c', 'd', 'e', 'f']

# Assuming correlation and p-value results are stored in variables like:
# precip_corr_p1, precip_pval_p1, precip_corr_p2, precip_pval_p2
# sst_corr_p1, sst_pval_p1, sst_corr_p2, sst_pval_p2
# var3_corr_p1, var3_pval_p1, var3_corr_p2, var3_pval_p2

# Replace with your actual variable names and results
correlation_results = {
    'SST': {'Period 1': {'corr': sst_p1_slope, 'pval': sst_p1_pval},
                      'Period 2': {'corr': sst_p2_slope, 'pval': sst_p2_pval}},
    'RSST':           {'Period 1': {'corr': rsst_p1_slope, 'pval': rsst_p1_pval},
                      'Period 2': {'corr': rsst_p2_slope, 'pval': rsst_p2_pval}},
    'CLLJ':    {'Period 1': {'corr': cllj_p1_slope, 'pval': cllj_p1_pval}, # Replace with your actual variable 3 results
                      'Period 2': {'corr': cllj_p2_slope, 'pval': cllj_p2_pval}}
}


# Define AxesGrid
grid = AxesGrid(fig, 111, nrows_ncols=(3, 2), axes_class=axes_class,
                share_all=True,
                cbar_location='right',
                cbar_mode='single',
                cbar_pad=0.1,
                cbar_size='3%',
                axes_pad=0.3,
                direction='row')

# Plotting loop
for i, ax in enumerate(grid):
    row = i // 2 # Determine row based on index (0, 1, 2)
    col = i % 2  # Determine column based on index (0 for Period 1, 1 for Period 2)
    variable = row_variables[row]
    period = f'Period {col + 1}'

    corr_data = correlation_results[variable][period]['corr']
    pval_data = correlation_results[variable][period]['pval']

    ax.set_extent(extent)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5, zorder=2, color='black')
    ax.grid(visible=True, lw=0.5, ls='--', color='lightgray')
    # ax.gridlines(draw_labels=True, linewidth=0.5, color='lightgray', alpha=0.5, linestyle='--')
    ax.set_xticks(np.arange(-90, -20, 10), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(10, 45, 10), crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.tick_params(axis='both', which='major', labelsize=10)

    im = ax.contourf(ds_rh700.longitude.values,
                    ds_rh700.latitude.values,
                    corr_data,
                    levels=levels_corr,
                    cmap=ncl_cm.temp_diff_18lev, # Using a diverging colormap for correlations
                    extend='both',
                    transform=ccrs.PlateCarree())

    # Hatching for significance (p < 0.05)
    # Create a masked array where values are masked if p_value >= 0.05
    masked_corr = np.ma.masked_where(pval_data >= 0.05, corr_data)

    ax.contourf(ds_rh700.longitude.values,
                ds_rh700.latitude.values,
                masked_corr,
                levels=levels_corr,
                hatches=['/////'], # Add hatching
                alpha=0, # Make the hatched area transparent
                transform=ccrs.PlateCarree())

    ax.text(0.02, 0.98, subplot_labels[i], transform=ax.transAxes, fontsize=12, fontweight='bold', va='top', ha='left')

    if i in [0, 2, 4]:
      ax.text(0.97, 0.98, variable, transform=ax.transAxes, fontsize=14, fontweight='bold', va='top', ha='right')

    if i in [0, 1]:
      ax.set_title(subplot_titles[i], loc='left', fontsize=12, fontweight='bold')


# Add a single colorbar for all subplots (assuming the same color scale)
cbar = grid.cbar_axes[0].colorbar(im)
cbar.set_label('Pearson Correlation', labelpad=14) # Updated label

plt.tight_layout()
plt.show()

## ---- Save figure ----
fig.savefig('../figures/figure5.pdf', dpi=300, format='pdf', bbox_inches='tight')